In [84]:
import nltk
import pandas as pd
import numpy as np
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
#### Load "derecha diario" dataset from csv.

In [85]:
df = pd.read_csv('./data/derecha_econ_news.csv')

In [86]:
df.shape

(339, 7)

In [87]:
df.head(5)

,Unnamed: 0,date,flyer,title,lead,body,url
0,0,30/11/2020,Economía,Economía española: en 12 meses se perdieron 10...,"Debido a la segunda ola del coronavirus, se pr...",Según el registro de la Seguridad Social españ...,https://derechadiario.com.ar/economia/economia...
1,1,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
2,2,29/11/2020,Economía,El reconocido ejecutivo Mark Mobius recomienda...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
3,3,28/11/2020,Economía,Ajuste a los jubilados: el aumento por decreto...,El Gobierno decretó un pobre aumento en torno ...,Alberto Fernández consolidó su reforma del sis...,https://derechadiario.com.ar/economia/ajuste-a...
4,4,27/11/2020,Economía,Brasil creó 395.000 puestos de trabajo en octu...,El Ministerio de Economía de Brasil confirmó u...,Con el acelerado ritmo de actividad en la econ...,https://derechadiario.com.ar/economia/brasil-c...


### Topics
Arbitrary selection of topics from news flyers of the portal's economic section.

In [88]:
exchange = df.title[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo')]
exchange.shape

(32,)

In [89]:
inflation = df.title[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec')]
inflation.shape

(38,)

### Corpus (Body)
Clipping of the selected news bodies

#### Topic: Inflation

In [90]:
inflation = df.loc[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec', na=False)]
inflation.shape

(38, 7)

In [91]:
inflation.date

1      29/11/2020
8      24/11/2020
20     12/11/2020
22     11/11/2020
24     09/11/2020
31     02/11/2020
41     24/10/2020
43     23/10/2020
54     15/10/2020
56     14/10/2020
58     12/10/2020
81     16/09/2020
88     11/09/2020
90     09/09/2020
97     01/09/2020
102    27/08/2020
113    14/08/2020
114    14/08/2020
132    21/07/2020
135    15/07/2020
165    12/06/2020
184    27/05/2020
190    22/05/2020
196    17/05/2020
198    15/05/2020
206    11/05/2020
210    08/05/2020
215    06/05/2020
221    01/05/2020
223    30/04/2020
226    28/04/2020
233    22/04/2020
241    18/04/2020
247    15/04/2020
266    06/04/2020
272    02/04/2020
298    17/03/2020
305    12/03/2020
Name: date, dtype: object

In [92]:
inflation.dropna(inplace=True)
inflation.shape

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(36, 7)

In [93]:
inflation = inflation.reset_index()

In [94]:
inflation.insert(loc=1, column='topics', value='inflation')

In [95]:
inflation.head(5)

,index,topics,Unnamed: 0,date,flyer,title,lead,body,url
0,1,inflation,1,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
1,8,inflation,8,24/11/2020,Economía,El economista Steve Hanke advierte que la infl...,El economista norteamericano de la Universidad...,"El economista Steve Hanke, profesor e investig...",https://derechadiario.com.ar/economia/el-econo...
2,20,inflation,20,12/11/2020,Economía,Se acelera el aumento de precios: la inflación...,El Índice de Precios al Consumidor aumentó un ...,El INDEC divulgó las cifras oficiales para el ...,https://derechadiario.com.ar/economia/se-acele...
3,22,inflation,22,11/11/2020,Economía,Socialismo y miseria: la economía venezolana c...,El régimen del dictador Maduro continúa destru...,Desde la toma ilegal del poder de Maduro hace ...,https://derechadiario.com.ar/economia/socialis...
4,24,inflation,24,09/11/2020,Economía,“Efecto Lipovetzky”: los alquileres siguen aum...,"La iniciativa del PRO, formulada originalmente...",En un año marcado por las nuevas regulaciones ...,https://derechadiario.com.ar/economia/efecto-l...


#### Topic: Currency Exchange

In [96]:
exchange = df.loc[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo', na=False)]
exchange.shape

(32, 7)

In [97]:
exchange.date

2      29/11/2020
8      24/11/2020
13     18/11/2020
18     13/11/2020
39     26/10/2020
42     23/10/2020
49     19/10/2020
55     15/10/2020
60     09/10/2020
63     07/10/2020
65     05/10/2020
68     02/10/2020
74     25/09/2020
82     16/09/2020
83     15/09/2020
90     09/09/2020
113    14/08/2020
128    25/07/2020
145    03/07/2020
166    11/06/2020
171    07/06/2020
181    29/05/2020
218    04/05/2020
227    27/04/2020
228    27/04/2020
232    23/04/2020
242    17/04/2020
243    17/04/2020
248    14/04/2020
267    04/04/2020
307    12/03/2020
331    23/02/2020
Name: date, dtype: object

In [98]:
exchange.dropna(inplace=True)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
exchange = exchange.reset_index()
exchange.shape

(31, 8)

In [100]:
exchange.insert(loc=1, column='topics', value='exchange')

In [101]:
exchange.head(5)

,index,topics,Unnamed: 0,date,flyer,title,lead,body,url
0,2,exchange,2,29/11/2020,Economía,El reconocido ejecutivo Mark Mobius recomienda...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
1,8,exchange,8,24/11/2020,Economía,El economista Steve Hanke advierte que la infl...,El economista norteamericano de la Universidad...,"El economista Steve Hanke, profesor e investig...",https://derechadiario.com.ar/economia/el-econo...
2,13,exchange,13,18/11/2020,Economía,Desesperación en el régimen chavista: el Banco...,"Tras casi cinco años de hiperinflación, el Ban...","Según un reciente informe de Bloomberg, fuente...",https://derechadiario.com.ar/economia/desesper...
3,18,exchange,18,13/11/2020,Economía,Nueva subida del dólar blue: ya acumula $23 en...,"Tras haber caído por debajo de los $150, el dó...",Hoy el dólar libre llegó a venderse a $174 en ...,https://derechadiario.com.ar/economia/nueva-su...
4,39,exchange,39,26/10/2020,Economía,El Gobierno lanza una mega-licitación de pesos...,El ministro Guzmán preparó una serie de instru...,Después de casi dos semanas de constantes aume...,https://derechadiario.com.ar/economia/el-gobie...


#### Save selection

In [102]:
data_clean = inflation.append(exchange)

In [103]:
pickle.dump(data_clean, open( "data/df_clean_der.pkl", "wb" ))

In [104]:
corpus = inflation.append(exchange)
corpus.shape

(67, 9)

In [105]:
corpus.sample(5)

,index,topics,Unnamed: 0,date,flyer,title,lead,body,url
19,135,inflation,135,15/07/2020,Economía,"Con la economía postrada, la inflación de juni...","Según el informe presentado hoy por el INDEC, ...",El Índice de Precios al Consumidor (IPC) que m...,https://derechadiario.com.ar/economia/con-la-e...
6,49,exchange,49,19/10/2020,Economía,El plan económico de Biden haría a todas las f...,Según las estimaciones de la Hoover Institutio...,La progresiva radicalización hacia la izquierd...,https://derechadiario.com.ar/economia/el-plan-...
25,210,inflation,210,08/05/2020,Economía,Broda advirtió que 2020 podría cerrar con tasa...,"En una entrevista para Radio Mitre, el famoso ...",En medio de la parálisis económica y las expec...,https://derechadiario.com.ar/economia/broda-ad...
2,13,exchange,13,18/11/2020,Economía,Desesperación en el régimen chavista: el Banco...,"Tras casi cinco años de hiperinflación, el Ban...","Según un reciente informe de Bloomberg, fuente...",https://derechadiario.com.ar/economia/desesper...
14,97,inflation,97,01/09/2020,Economía,"Página 12, el kirchnerismo y el terraplanismo ...",Tras el reciente comportamiento del nivel de p...,"Recientemente, el diario ultra kirchnerista Pá...",https://derechadiario.com.ar/economia/pagina-1...


I drop those news labeled under both topics

In [34]:
corpus = corpus.drop_duplicates(subset='index',keep=False)

In [35]:
corpus.shape

(61, 9)

### Tokenization
#### Manual

In [36]:
import re
import unidecode

"Stop words", special characters, accents and numbers are removed

In [37]:
stopwords_sp = stopwords.words('spanish')

In [38]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [39]:
tokenized = [tokenizer(documento) for documento in corpus.body]

In [40]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [41]:
len(vocabulario)

4699

#### CountVectorizer

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

Removal of special characters function to load in vectorizer

In [73]:
def removal(text):
    text = re.sub(r'(\d|\$|\%|\+)', '', text.lower())
    return re.sub(r'\d+', '', text)

In [74]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode', preprocessor=removal)

In [75]:
vectorizer.fit(corpus.body)

CountVectorizer(preprocessor=<function removal at 0x7f3bb5f4b050>,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [76]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(4549, 4549)

I prefare the manual tokenization results since the process is clearer and there are more terms.

### Pickle

Manual tokens

In [77]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [78]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [79]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [80]:
# add classification column
topics = ['inflation' for i in range((corpus.topics == 'inflation').sum())] + ['exchange' for i in range((corpus.topics == 'exchange').sum())]
token_df.insert(loc=0, column='topics', value=topics)

In [81]:
token_df

,topics,abajo,abandonara,abandone,abastecimiento,abatida,abierta,abierto,aboga,abonados,...,woyecheszen,xx,yaguarete,york,ypf,yuanes,zamora,zona,zonaprop,zonas
doc1,inflation,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
doc2,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
doc3,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,0
doc5,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc57,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc58,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc59,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc60,exchange,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
token_df.to_pickle('tokens/topic_tokens_der.pkl')

#### CV tokens

In [83]:
pickle.dump(vectorizer, open( "tokens/cv_der.pkl", "wb" ))